## 환경 설정

In [28]:
from dotenv import load_dotenv

load_dotenv()

True

### Callback 정의

In [22]:
from langfuse.langchain import CallbackHandler

langfuse_handler = CallbackHandler()

In [23]:
import time
import logging
from datetime import datetime
from typing import Dict, List, Any, Optional
from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.outputs import LLMResult

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

class PerformanceMonitoringCallback(BaseCallbackHandler):
    """LLM 호출 성능을 모니터링하는 콜백 핸들러"""
    
    def __init__(self):
        self.start_time: Optional[float] = None
        self.token_usage: Dict[str, Any] = {}
        self.call_count: int = 0
        
    def on_llm_start(
        self, 
        serialized: Dict[str, Any], 
        prompts: List[str], 
        **kwargs: Any
    ) -> None:
        """LLM 호출이 시작될 때 호출"""
        self.start_time = time.time()
        self.call_count += 1
        print(f"🚀 LLM 호출 #{self.call_count} 시작 - {datetime.now().strftime('%H:%M:%S')}")
        
        # 첫 번째 프롬프트의 길이 확인
        if prompts:
            print(f"📝 프롬프트 길이: {len(prompts[0])} 문자")
        
    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> None:
        """LLM 호출이 완료될 때 호출"""
        if self.start_time:
            duration = time.time() - self.start_time
            print(f"✅ LLM 호출 완료 - 소요시간: {duration:.2f}초")
            
            # 토큰 사용량 추적
            if response.generations:
                generation = response.generations[0][0]
                
                # usage_metadata를 우선 확인 
                if hasattr(generation, 'usage_metadata') and generation.usage_metadata:
                    usage = generation.usage_metadata
                    print(f"🔢 토큰 사용량: {usage}")
                    self.token_usage = usage
                    
                # 구버전 호환성을 위한 llm_output 확인
                elif hasattr(response, 'llm_output') and response.llm_output:
                    usage = response.llm_output.get('token_usage', {})
                    if usage:
                        print(f"🔢 토큰 사용량: {usage}")
                        self.token_usage = usage
                        
                # 응답 길이 체크
                if hasattr(generation, 'text'):
                    response_text = generation.text
                    print(f"📊 응답 길이: {len(response_text)} 문자")
        
    def on_llm_error(self, error: Exception, **kwargs: Any) -> None:
        """LLM 호출에서 오류가 발생할 때 호출"""
        print(f"❌ LLM 호출 오류: {str(error)}")
        
    def get_statistics(self) -> Dict[str, Any]:
        """현재까지의 통계 정보를 반환"""
        return {
            "total_calls": self.call_count,
            "last_token_usage": self.token_usage
        }

## 1. 입력 데이터 정의

In [18]:
from enum import Enum, StrEnum, auto
from datetime import datetime

class Purpose(StrEnum):
    """Main purpose of the trip."""
    Healing = auto()         # Relaxation, rest
    Activity = auto()        # Physical activities, outdoor adventures
    Culture = auto()         # Cultural exploration, museums, heritage
    Shopping = auto()        # Shopping-focused travel
    Food = auto()            # Culinary-focused travel
    Nature = auto()          # Enjoying natural scenery
    Romance = auto()         # Romantic getaway
    Photography = auto()     # Photography-centric travel

class Companion(StrEnum):
    """Type of travel companion."""
    Solo = auto()            # Traveling alone
    Couple = auto()          # Traveling with a partner
    Family = auto()          # Family travel
    Friends = auto()         # Trip with friends

class Transport(StrEnum):
    """Preferred transport method."""
    Car = auto()             # Private vehicle
    PublicTransport = auto() # Bus, subway, train, etc.
    Walking = auto()         # On foot
    Bike = auto()            # Bicycle or similar

class PerPersonBudget(Enum):
    """Estimated budget per person in KRW."""
    ULTRA_BUDGET = (0, 50000, "KRW")          # 0 ~ 50,000 KRW
    BUDGET = (50001, 150000, "KRW")           # 50,001 ~ 150,000 KRW
    STANDARD = (150001, 300000, "KRW")        # 150,001 ~ 300,000 KRW
    PREMIUM = (300001, 500000, "KRW")         # 300,001 ~ 500,000 KRW
    LUXURY = (500001, float("inf"), "KRW")    # 500,001 KRW and above

    def __init__(self, min_price, max_price, currency):
        self.min_price = min_price
        self.max_price = max_price
        self.currency = currency

class SeasonPreference(StrEnum):
    """Preferred season for travel."""
    Spring = auto()
    Summer = auto()
    Autumn = auto()
    Winter = auto()
    NoPreference = auto()

class ActivityLevel(StrEnum):
    """Preferred activity level."""
    Light = auto()      # Mostly rest
    Moderate = auto()   # Balanced
    Active = auto()     # High movement / long walking

class UniqueExperience(StrEnum):
    """Optional unique or niche travel experiences."""
    TempleStay = auto()         # 사찰 체험, 템플스테이
    LocalFestival = auto()      # 지역 축제 참여
    NightView = auto()          # 야경 감상
    HotSpring = auto()          # 온천
    Island = auto()             # 섬 여행
    HanokStay = auto()          # 한옥 숙박
    HistoricalExperience = auto()  # 조선 시대 복식체험 등
    HealingWorkshop = auto()    # 명상, 요가, 향 체험 등
    Camping = auto()            # 캠핑, 글램핑 등
    KPopHallyu = auto()         # K-POP, 한류 팬 투어
    Other = auto()              # 기타 (자유 입력)


> 입력 예시

In [42]:
user_profile_01 = {
    "purpose": Purpose.Nature.value,
    "companion": Companion.Family.value,
    "num_people": 3,
    "has_pet": False,
    "has_infant": True,
    "transport": Transport.Car.value,
    "per_person_budget": PerPersonBudget.STANDARD.value,
    "target_location": [("51", "230")],
    "start_time": "20250801 09:00",
    "end_time": "20250804 16:00",
    "duration_days": 3,
    "prefer_dense_schedule": False,
    "season_preference": SeasonPreference.Summer.value,
    "activity_level": ActivityLevel.Moderate.value,
    "unique_experiences": [
        UniqueExperience.HotSpring.value,
        UniqueExperience.NightView.value,
        UniqueExperience.Other.value
    ],
    "custom_experience_note": "아이와 함께 체험할 수 있는 자연놀이터",
    "additional_notes": "아이랑 같이 쉬기 좋은 숙소 위주"
}
user_profile_02 = {
    "purpose": Purpose.Healing.value,
    "companion": Companion.Couple.value,
    "num_people": 2,
    "has_pet": False,
    "has_infant": False,
    "transport": Transport.PublicTransport.value,
    "per_person_budget": PerPersonBudget.PREMIUM.value,
    "target_location": [("50", "110"), ("50", "130")],
    "start_time": "20250910 15:00",
    "end_time": "20250911 20:00",
    "duration_days": 2,
    "prefer_dense_schedule": False,
    "season_preference": SeasonPreference.Autumn.value,
    "activity_level": ActivityLevel.Light.value,
    "unique_experiences": [
        UniqueExperience.NightView.value,
        UniqueExperience.Other.value
    ],
    "custom_experience_note": "별을 볼 수 있는 조용한 야외 공간",
    "additional_notes": "혼잡하지 않은 한적한 곳 선호"
}

user_profile_03 = {
    "purpose": Purpose.Culture.value,
    "companion": Companion.Family.value,
    "num_people": 7,
    "has_pet": False,
    "has_infant": True,
    "transport": Transport.Car.value,
    "per_person_budget": PerPersonBudget.STANDARD.value,
    "target_location": [("47", "130")],
    "start_time": "20251003 08:30",
    "end_time": "20251005 18:00",
    "duration_days": 3,
    "prefer_dense_schedule": False,
    "season_preference": SeasonPreference.Autumn.value,
    "activity_level": ActivityLevel.Light.value,
    "unique_experiences": [
        UniqueExperience.TempleStay.value,
        UniqueExperience.HistoricalExperience.value
    ],
    "custom_experience_note": "",
    "additional_notes": "아이와 어르신 모두 즐길 수 있는 전통문화 체험이 필요합니다. 편안한 숙소와 쉬는 시간 많은 일정이 좋습니다."
}

## 2. 카테고리 조회를 위한 Text2SQL 에이전트 정의

### 1\) 카테고리 벡터화

- 사용자 맞춤형 관광지를 의미 및 키워드 기반으로 탐색하기 위한 작업임

In [58]:
import ast
import re

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

from langchain_community.utilities import SQLDatabase
from langchain_community.retrievers import BM25Retriever

from langchain.retrievers import EnsembleRetriever

from kiwipiepy import Kiwi


def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))

db = SQLDatabase.from_uri(
    "sqlite:///../kto_data.db"
)

# 벡터화 작업 리스트
# cat_1 = query_as_list(
#     db, 
#     "SELECT DISTINCT lclsSystm1Nm 종목명 FROM lclsSystmCode2"
# )
cat_2 = query_as_list(
    db, 
    "SELECT DISTINCT lclsSystm2Nm 종목명 FROM lclsSystmCode2"
)
cat_3 = query_as_list(
    db, 
    "SELECT DISTINCT lclsSystm3Nm 종목명 FROM lclsSystmCode2"
)

# 임베딩 모델 정의
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 임베딩 벡터 저장소 및 검색기 생성
chroma_db = Chroma(
    collection_name="category",
    embedding_function=embeddings,
    persist_directory="../vs",
)
data_add = False # 데이터를 벡터스토어에 저장 여부 * 나중에 언어별 추가 필요
data = list(set(cat_2 + cat_3))
kiwi = Kiwi()

if data_add:
    # 데이터를 벡터스토어에 저장
    chroma_db.add_texts(data)       

retriever = chroma_db.as_retriever(
    search_kwargs={"k": 10}
)
bm25_retriever = BM25Retriever.from_texts(
    texts=data,
    preprocess_func=lambda x: [t.form for t in kiwi.tokenize(x)]
)
ensemble_retriever = EnsembleRetriever(
    retrievers=[retriever, bm25_retriever], 
    weights=[0.5, 0.5]          
)

In [59]:
import json

result_01 = ensemble_retriever.invoke(json.dumps(user_profile_01))
result_01

[Document(id='7cc7b3d0-db22-47c6-888e-dc8a5fafb142', metadata={}, page_content='농.산.어촌 체험'),
 Document(metadata={}, page_content='산, 고개, 오름, 봉우리'),
 Document(id='14bc465e-038b-45f0-a0d5-d2c127154436', metadata={}, page_content='행사'),
 Document(metadata={}, page_content='고분, 능'),
 Document(id='c8b34fad-7f9a-476c-9a5b-f40876e98688', metadata={}, page_content='행사시설'),
 Document(metadata={}, page_content='피자, 햄버거, 샌드위치 및 유사음식'),
 Document(id='bc8d2036-653b-4cbd-b31e-e76d1035980c', metadata={}, page_content='기타행사'),
 Document(metadata={}, page_content='골목길, 문화거리'),
 Document(id='220efeaf-74eb-4eb6-a136-0cf0e670ec3f', metadata={}, page_content='생태관광지'),
 Document(id='1e6ab10f-89b9-46f6-aead-e5a5e76770d3', metadata={}, page_content='모텔'),
 Document(id='9e76ce47-52fa-413b-be47-cee6709ccec5', metadata={}, page_content='동물원'),
 Document(id='a2908c28-89e4-405a-acb9-d2c1fab0392d', metadata={}, page_content='기타자연생태'),
 Document(id='4e2d1ff0-9aa0-41ff-9d9a-181a686e2036', metadata={}, page_content='기

In [60]:
result_02 = ensemble_retriever.invoke(json.dumps(user_profile_02))
result_02

[Document(id='14bc465e-038b-45f0-a0d5-d2c127154436', metadata={}, page_content='행사'),
 Document(metadata={}, page_content='산, 고개, 오름, 봉우리'),
 Document(id='c8b34fad-7f9a-476c-9a5b-f40876e98688', metadata={}, page_content='행사시설'),
 Document(metadata={}, page_content='고분, 능'),
 Document(id='bc8d2036-653b-4cbd-b31e-e76d1035980c', metadata={}, page_content='기타행사'),
 Document(metadata={}, page_content='피자, 햄버거, 샌드위치 및 유사음식'),
 Document(id='1e6ab10f-89b9-46f6-aead-e5a5e76770d3', metadata={}, page_content='모텔'),
 Document(metadata={}, page_content='골목길, 문화거리'),
 Document(id='1619be02-68d9-4780-aafd-e29aecb4873b', metadata={}, page_content='유람선/잠수함관광'),
 Document(id='7cc7b3d0-db22-47c6-888e-dc8a5fafb142', metadata={}, page_content='농.산.어촌 체험'),
 Document(id='09dc04fc-0f81-43eb-b638-976611469c4a', metadata={}, page_content='관광식당'),
 Document(id='9e76ce47-52fa-413b-be47-cee6709ccec5', metadata={}, page_content='동물원'),
 Document(id='ecd6f8ae-6c0f-4c71-af5e-8eeac098f921', metadata={}, page_content=

In [61]:
result_03 = ensemble_retriever.invoke(json.dumps(user_profile_03))
result_03

[Document(id='14bc465e-038b-45f0-a0d5-d2c127154436', metadata={}, page_content='행사'),
 Document(metadata={}, page_content='산, 고개, 오름, 봉우리'),
 Document(id='7cc7b3d0-db22-47c6-888e-dc8a5fafb142', metadata={}, page_content='농.산.어촌 체험'),
 Document(metadata={}, page_content='고분, 능'),
 Document(id='c8b34fad-7f9a-476c-9a5b-f40876e98688', metadata={}, page_content='행사시설'),
 Document(metadata={}, page_content='피자, 햄버거, 샌드위치 및 유사음식'),
 Document(id='bc8d2036-653b-4cbd-b31e-e76d1035980c', metadata={}, page_content='기타행사'),
 Document(metadata={}, page_content='골목길, 문화거리'),
 Document(id='1e6ab10f-89b9-46f6-aead-e5a5e76770d3', metadata={}, page_content='모텔'),
 Document(id='46001330-3bd1-482b-a818-9542e16b57af', metadata={}, page_content='한국문화원'),
 Document(id='9989f801-5dee-40a3-8977-1f2b8dc1bdd5', metadata={}, page_content='문화관광축제'),
 Document(id='c7141552-758c-4b70-9c4c-ce91128833f9', metadata={}, page_content='자연경관(하천‧해양)'),
 Document(id='09dc04fc-0f81-43eb-b638-976611469c4a', metadata={}, page_co

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
)

model.invoke("Hello")

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--a55fa48c-2331-49c6-b86d-2442b085aa49-0', usage_metadata={'input_tokens': 2, 'output_tokens': 9, 'total_tokens': 234, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 223}})